**AI & Machine Learning (KAN-CINTO4003U) - Copenhagen Business School | Spring 2025**

***

# Part II: Agents

Please see the description of the assignment in the README file (section 1) <br>
**Guide notebooks**: [guides/router_agents_guide.ipynb](guides/agents_guide.ipynb) and [guides/tool_agents_guide.ipynb](guides/tool_agents_guide.ipynb)


***
<br>

* Play around with the agent systems in the guides. Maybe try to create a similar system with new nodes? Or you can try to add new tools to the existing tool-using agent system.

* Remember to include some reflections on your results. Are there, for example, any hyperparameters that are particularly important?

* You should follow the steps given in the `router_agents_guide` notebook or the `tool_agents_guide` notebook to create your own agent system - or try to optimize the existing ones.

<br>

***

## Imports

In [25]:
#Import
import os
import requests
import streamlit as st
from decouple import config

# LangChain / LangGraph imports
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from langchain_ibm import ChatWatsonx

## Call the API

In [ ]:
# Load API keys from .env or environment
WX_API_KEY = config("WX_API_KEY")
WX_PROJECT_ID = config("WX_PROJECT_ID")
WX_API_URL = "https://us-south.ml.cloud.ibm.com"
os.environ["OPENWEATHERMAP_API_KEY"] = config("OPENWEATHERMAP_API_KEY")



## Creating the weather tool

In [ ]:
@tool
def get_current_weather(city: str = "Copenhagen", units: str = "metric"):
    """
    Retrieves the current weather data for a specified city.
    """
    api_key = os.environ.get("OPENWEATHERMAP_API_KEY")

    response = requests.get(
        "http://api.openweathermap.org/data/2.5/weather",
        params={
            "q": city,
            "appid": api_key,
            "units": units
        }
    )

    data = response.json()

    if response.status_code != 200 or "main" not in data:
        return {
            "error": f"Unable to retrieve weather for {city}",
            "details": data.get("message", "No additional information available.")
        }

    return {
        "city": data.get("name", city),
        "temperature": data["main"]["temp"],
        "weather": data["weather"][0]["main"],
        "humidity": data["main"]["humidity"],
        "wind_speed": data["wind"]["speed"],
        "forecast": data["weather"][0]["description"],
        "icon": data["weather"][0]["icon"],
        "weather_tomorrow": data["main"]["temp_max"],
        "weather_yesterday": data["main"]["temp_min"],
        "rain": data.get("rain", {}).get("1h", 0),
        "snow": data.get("snow", {}).get("1h", 0),
    }


# Create the WatsonX LLM
chat_model = ChatWatsonx(
    url=WX_API_URL,
    apikey=WX_API_KEY,
    project_id=WX_PROJECT_ID,
    model_id="mistralai/mistral-large",
)


In [ ]:
# Define tool list
tools = [get_current_weather]

# Build the agent
# We turned the debug flag off to avoid printing the entire conversation in the console.
graph = create_react_agent(chat_model, tools=tools, debug=False)

# Ask a question
response = graph.invoke(
    {
        
        "messages": [
            ("user", "Whats the weather in Aarhus?"),
        ]
    },
    debug=False
)

# Print the final AI message
print(response["messages"][-1].content)

 In Aarhus now is:
A temperature of 10.37°C with partly cloudy, a humidity of 90% and a wind speed of 2.5km/h.
Yesterday the temperature was 9.22°C and tomorrow it will be 11.12°C.
